# Sparse Operation Kit #
---
This notebook introduces what is sparse operation kit and how to use it to accerlerate the recommander system's training process.

Sparse Opertion Kit (hereafter SOK) is a toolkit aiming at wrapping those effecient algorithms / implementations used in recommendation scenarios, which includes many sparse operations, into a user-friendly library. When user wants to leverage those GPU-accerlerated algorithms to speed up their application, they can quickly start from this Python toolkit. 

## Documents ##
Documentation: https://nvidia-merlin.github.io/HugeCTR/sparse_operation_kit/master/index.html

## Menu ##
1. **Installation**
2. **Single-node, Multi-GPUs synchronized training**
3. **Multi-node, Multi-GPUs synchronized training**

### Installation ###
+ **Requirements**
    - TensorFlow 2.x


+ **Get SOK from NGC** <br>
The SparseOperationKit is preinstalled in the [Merlin Tensorflow Training Container](https://ngc.nvidia.com/catalog/containers/nvidia:merlin:merlin-tensorflow-training): `nvcr.io/nvidia/merlin/merlin-tensorflow-training:22.04`. <br>
You can check the existence of required libraries by running the following Python code after launching this container.
```shell
$ python3 -c "import sparse_operation_kit as sok"
```

+ **Build SOK from Souce Code** <br>
If you want to build SparseOperationKit from the souce code instead of using the NGC container, please refer to the [Setup development environment](../../docs/hugectr_contributor_guide.md#build-sparse-operation-kit-sok-from-source).

### Single-node, Multi-GPUs synchronized training ###

Firstly, specify hyper parameters.

In [1]:
%reset -f

args = dict()

args["gpu_num"] = 8                               # the number of available GPUs
args["iter_num"] = 50                             # the number of training iteration
args["max_vocabulary_size_per_gpu"] = 1024
args["slot_num"] = 10                             # the number of feature fields in this embedding layer
args["max_nnz"] = 4                               # the maximum number of valid features in each slot
args["embedding_vec_size"] = 4                    # the dimension of embedding vectors
args["combiner"] = "mean"                         # the reduction combiner used intra slots, it can be [mean, sum]
args["global_batch_size"] = 65536                 # the globally batchsize for all GPUs
args["optimizer"] = "plugin_adam"                 # the optimizer used for training, it can be [plugin_adam, adam, sgd]

Secondly, import the used modules.

In [2]:
import sys, os, json
import sparse_operation_kit as sok
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"] = ",".join(map(str, range(args["gpu_num"])))
import numpy as np

[INFO]: sparse_operation_kit is imported


Thirdly, define a DNN model using TensorFlow.

In [3]:
class TfDemo(tf.keras.models.Model):
    def __init__(self, 
                 init_tensors, 
                 combiner, 
                 global_batch_size,
                 slot_num, 
                 embedding_vec_size,
                 **kwargs):
        super(TfDemo, self).__init__(**kwargs)
        self.combiner = combiner
        self.global_batch_size = global_batch_size
        self.slot_num = slot_num
        self.embedding_vec_size = embedding_vec_size

        self.init_tensors = init_tensors
        self.params = tf.Variable(initial_value=tf.concat(self.init_tensors, axis=0))

        self.dense_layer = tf.keras.layers.Dense(units=1, activation=None,
                                                 kernel_initializer="ones",
                                                 bias_initializer="zeros")

    def call(self, inputs, training=True):
        # [batchsize * slot_num, embedding_vec_size]
        embedding_vector = tf.nn.embedding_lookup_sparse(params=self.params, sp_ids=inputs,
                                                        sp_weights=None, combiner=self.combiner)

        # [batchsize, slot_num * embedding_vec_size]
        embedding_vector = tf.reshape(embedding_vector, shape=[self.global_batch_size, self.slot_num * self.embedding_vec_size])
        logit = self.dense_layer(embedding_vector)
        return logit, embedding_vector

Fourthly, define the same DNN model using SOK.

In [4]:
class SOKDemo(tf.keras.models.Model):
    def __init__(self,
                 combiner,
                 max_vocabulary_size_per_gpu,
                 slot_num,
                 max_nnz,
                 embedding_vec_size, 
                 **kwargs):
        super(SOKDemo, self).__init__(**kwargs)

        self.combiner = combiner
        self.max_vocabulary_size_per_gpu = max_vocabulary_size_per_gpu
        self.slot_num = slot_num
        self.max_nnz = max_nnz
        self.embedding_vec_size = embedding_vec_size

        self.embedding_layer = sok.DistributedEmbedding(combiner=self.combiner,
                                                           max_vocabulary_size_per_gpu=self.max_vocabulary_size_per_gpu,
                                                           embedding_vec_size=self.embedding_vec_size,
                                                           slot_num=self.slot_num,
                                                           max_nnz=self.max_nnz)

        self.dense_layer = tf.keras.layers.Dense(units=1, activation=None,
                                                 kernel_initializer="ones",
                                                 bias_initializer="zeros")

    def call(self, inputs, training=True):
        # [batchsize, slot_num, embedding_vec_size]
        embedding_vector = self.embedding_layer(inputs, training=training)
        # [batchsize, slot_num * embedding_vec_size]
        embedding_vector = tf.reshape(embedding_vector, shape=[-1, self.slot_num * self.embedding_vec_size])
        # [batchsize, 1]
        logit = self.dense_layer(embedding_vector)
        return logit, embedding_vector

Fifthly, generate synthetic dataset and initial values that is used to initialize embedding parameters.

In [5]:
# import utility python script
sys.path.append("../unit_test/test_scripts/tf2/")
import utils

In [6]:
# -1 is used to represent the invalid keys
random_samples = utils.generate_random_samples(num_of_samples=args["global_batch_size"] * args["iter_num"],
                                               vocabulary_size=args["gpu_num"] * args["max_vocabulary_size_per_gpu"],
                                               slot_num=args["slot_num"],
                                               max_nnz=args["max_nnz"])

[INFO]: begin to generate random samples
[INFO]: generated random samples


In [7]:
# check ramdom_samples
random_samples

(array([[[ 237,   38,   -1,   -1],
         [1255,  921,   -1,   -1],
         [2139,   -1,   -1,   -1],
         ...,
         [  -1,   -1,   -1,   -1],
         [7018, 6987,   -1,   -1],
         [7875,   -1,   -1,   -1]],
 
        [[ 749,  718,  680,  642],
         [1606,  894,   -1,   -1],
         [1859, 1782,   -1,   -1],
         ...,
         [6455, 6384, 6321,   -1],
         [6589,   -1,   -1,   -1],
         [7890,   -1,   -1,   -1]],
 
        [[ 653,  582,   -1,   -1],
         [1063,  929,  858,   -1],
         [1953,   -1,   -1,   -1],
         ...,
         [6209, 6084, 5866, 5741],
         [6942,   -1,   -1,   -1],
         [7886, 7410,   -1,   -1]],
 
        ...,
 
        [[ 166,   -1,   -1,   -1],
         [1580, 1210, 1058,  913],
         [2326, 2174, 1796,   -1],
         ...,
         [5917,   -1,   -1,   -1],
         [7227, 7066, 6738,   -1],
         [8056,   -1,   -1,   -1]],
 
        [[ 356,   -1,   -1,   -1],
         [1346, 1177,   -1,   -1],
       

In [8]:
# generate initial value for embedding parameters
init_tensors = utils.get_ones_tensor(max_vocab_size_per_gpu=args["max_vocabulary_size_per_gpu"],
                                     embedding_vec_size=args["embedding_vec_size"],
                                     num=args["gpu_num"])

In [9]:
# check init_tensors
init_tensors

[array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        ...,
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32),
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        ...,
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32),
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        ...,
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32),
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        ...,
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32),
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        ...,
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32),
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1

Sixly, define training loop for TensorFlow and SOK

In [10]:
def test_tf_demo(args, init_tensors, *random_samples):
    dataset = utils.tf_dataset(*random_samples, batchsize=args["global_batch_size"], to_sparse_tensor=True, repeat=1)

    loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)

    tf_demo = TfDemo(init_tensors, args["combiner"], args["global_batch_size"], 
                     args["slot_num"], args["embedding_vec_size"])

    optimizer = utils.get_dense_optimizer(args["optimizer"])(learning_rate=0.1)

    @tf.function
    def _train_step(inputs, labels):
        with tf.GradientTape() as tape:
            logit, embedding_vector = tf_demo(inputs, training=True)
            loss = loss_fn(labels, logit)
        grads = tape.gradient(loss, tf_demo.trainable_variables)
        optimizer.apply_gradients(zip(grads, tf_demo.trainable_variables))
        return logit, embedding_vector

    tf_results = list()

    for i, (sparse_tensors, labels) in enumerate(dataset):
        print("-"*30, str(i), "-"*30)
        logit, embedding_vector = _train_step(sparse_tensors, labels)
        print("[INFO]: embedding_vector:\n", embedding_vector)
        tf_results.append(embedding_vector)

        # FIXME: because plugin sleepd, here is only used for 
        # simulate the same DNN structure. 
        import time
        time.sleep(0.2) # seconds

    return tf_results

In [11]:
def test_sok_demo(args, init_tensors, *random_samples):
    strategy = tf.distribute.MirroredStrategy()
    with strategy.scope():
        result = sok.Init(global_batch_size=args["global_batch_size"])

        plugin_demo = SOKDemo(combiner=args["combiner"], 
                                 max_vocabulary_size_per_gpu=args["max_vocabulary_size_per_gpu"],
                                 slot_num=args["slot_num"], max_nnz=args["max_nnz"],
                                 embedding_vec_size=args["embedding_vec_size"])

        emb_opt = utils.get_embedding_optimizer(args["optimizer"])(learning_rate=0.1)
        dense_opt = utils.get_dense_optimizer(args["optimizer"])(learning_rate=0.1)

    plugin_saver = sok.Saver()

    plugin_saver.load_embedding_values(plugin_demo.embedding_layer.embedding_variable, init_tensors)

    loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
    def _replica_loss(labels, logits):
        loss = loss_fn(labels, logits)
        return tf.nn.compute_average_loss(loss, global_batch_size=args["global_batch_size"])

    @tf.function
    def _train_step(inputs, labels):
        with tf.GradientTape() as tape:
            logit, embedding_vector = plugin_demo(inputs, training=True)
            loss = _replica_loss(labels, logit)
        embedding_variables, other_variable = sok.split_embedding_variable_from_others(plugin_demo.trainable_variables)
        grads, emb_grads = tape.gradient(loss, [other_variable, embedding_variables])
        if 'plugin' not in args["optimizer"]:
            with sok.OptimizerScope(embedding_variables):
                emb_opt.apply_gradients(zip(emb_grads, embedding_variables),
                                        experimental_aggregate_gradients=False)
        else:
            emb_opt.apply_gradients(zip(emb_grads, embedding_variables),
                                    experimental_aggregate_gradients=False)
        dense_opt.apply_gradients(zip(grads, other_variable))
        return logit, embedding_vector

    sok_results = list()

    def _dataset_fn(input_context):
        replica_batch_size = input_context.get_per_replica_batch_size(args["global_batch_size"])
        dataset = utils.tf_dataset(*random_samples, batchsize=replica_batch_size, to_sparse_tensor=True, repeat=1)
        dataset = dataset.shard(input_context.num_input_pipelines, input_context.input_pipeline_id)
        return dataset

    dataset = strategy.distribute_datasets_from_function(_dataset_fn)
    
    for i, (sparse_tensors, replica_labels) in enumerate(dataset):
        print("-" * 30, "step ", str(i), "-" * 30)
        logit, embedding_vector = strategy.run(_train_step, args=(sparse_tensors, replica_labels))
        print("[INFO]: embedding_vector\n", embedding_vector)
        sok_results.append(embedding_vector)

        # FIXME: when the forward computation is too fast, there
        # may exist some conficts with datareader, which cause the program hang.
        import time
        time.sleep(0.2) # seconds

    return sok_results

Sevenly, start training process. Compare whether the embedding vectors obtained from TensorFlow and SOK are consistent in all iterations.

In [12]:
# train TensorFlow Demo Model, this command will print each iteration's embedding vector
tf_results = test_tf_demo(args, init_tensors, *random_samples)

2021-12-07 04:48:26.388375: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2021-12-07 04:48:26.388450: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13631 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:06:00.0, compute capability: 7.0
2021-12-07 04:48:26.390576: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2021-12-07 04:48:26.390609: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14631 MB memory:  -> device: 1, name: Tesla V100-SXM2-16GB, pci bus id: 0000:07:00.0, compute capability: 7.0
2021-12-07 04:48:26.392713: W tensorflow/cor

------------------------------ 0 ------------------------------
[INFO]: embedding_vector:
 tf.Tensor(
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]], shape=(65536, 40), dtype=float32)
------------------------------ 1 ------------------------------
[INFO]: embedding_vector:
 tf.Tensor(
[[0.9005389  0.9005389  0.9005389  ... 0.9004773  0.9004773  0.9004773 ]
 [0.9004776  0.9004776  0.9004776  ... 0.90067196 0.90067196 0.90067196]
 [0.90046406 0.90046406 0.90046406 ... 0.9004576  0.9004576  0.9004576 ]
 ...
 [0.9005024  0.9005024  0.9005024  ... 0.9005311  0.9005311  0.9005311 ]
 [0.9005841  0.9005841  0.9005841  ... 0.90050334 0.90050334 0.90050334]
 [0.90044    0.90044    0.90044    ... 0.90049696 0.90049696 0.90049696]], shape=(65536, 40), dtype=float32)
------------------------------ 2 ------------------------------
[INFO]: embedding_vector:
 tf.Tensor(
[[0.8012184  0.8012184  

------------------------------ 15 ------------------------------
[INFO]: embedding_vector:
 tf.Tensor(
[[-0.15322745 -0.15322745 -0.15322745 ... -0.17870769 -0.17870769
  -0.17870769]
 [-0.15844432 -0.15844432 -0.15844432 ... -0.17112882 -0.17112882
  -0.17112882]
 [-0.14272878 -0.14272878 -0.14272878 ... -0.18848768 -0.18848768
  -0.18848768]
 ...
 [-0.17903242 -0.17903242 -0.17903242 ... -0.21368095 -0.21368095
  -0.21368095]
 [-0.18603516 -0.18603516 -0.18603516 ... -0.18102147 -0.18102147
  -0.18102147]
 [-0.16306126 -0.16306126 -0.16306126 ... -0.18238421 -0.18238421
  -0.18238421]], shape=(65536, 40), dtype=float32)
------------------------------ 16 ------------------------------
[INFO]: embedding_vector:
 tf.Tensor(
[[-0.22411308 -0.22411308 -0.22411308 ... -0.19928825 -0.19928825
  -0.19928825]
 [-0.20470376 -0.20470376 -0.20470376 ... -0.19559422 -0.19559422
  -0.19559422]
 [-0.19532926 -0.19532926 -0.19532926 ... -0.22929913 -0.22929913
  -0.22929913]
 ...
 [-0.21242538 -0.21

------------------------------ 29 ------------------------------
[INFO]: embedding_vector:
 tf.Tensor(
[[-0.15862423 -0.15862423 -0.15862423 ... -0.17314598 -0.17314598
  -0.17314598]
 [-0.1752002  -0.1752002  -0.1752002  ... -0.17943908 -0.17943908
  -0.17943908]
 [-0.20004535 -0.20004535 -0.20004535 ... -0.15819792 -0.15819792
  -0.15819792]
 ...
 [-0.17449039 -0.17449039 -0.17449039 ... -0.17067263 -0.17067263
  -0.17067263]
 [-0.19266994 -0.19266994 -0.19266994 ... -0.17156011 -0.17156011
  -0.17156011]
 [-0.20928861 -0.20928861 -0.20928861 ... -0.13899939 -0.13899939
  -0.13899939]], shape=(65536, 40), dtype=float32)
------------------------------ 30 ------------------------------
[INFO]: embedding_vector:
 tf.Tensor(
[[-0.17543076 -0.17543076 -0.17543076 ... -0.1694403  -0.1694403
  -0.1694403 ]
 [-0.17624065 -0.17624065 -0.17624065 ... -0.19092642 -0.19092642
  -0.19092642]
 [-0.17132854 -0.17132854 -0.17132854 ... -0.15731235 -0.15731235
  -0.15731235]
 ...
 [-0.15872258 -0.158

------------------------------ 43 ------------------------------
[INFO]: embedding_vector:
 tf.Tensor(
[[-0.26315174 -0.26315174 -0.26315174 ... -0.20632924 -0.20632924
  -0.20632924]
 [-0.19329208 -0.19329208 -0.19329208 ... -0.20625876 -0.20625876
  -0.20625876]
 [-0.23012948 -0.23012948 -0.23012948 ... -0.24424371 -0.24424371
  -0.24424371]
 ...
 [-0.2262593  -0.2262593  -0.2262593  ... -0.22234231 -0.22234231
  -0.22234231]
 [-0.20353846 -0.20353846 -0.20353846 ... -0.20778775 -0.20778775
  -0.20778775]
 [-0.19702505 -0.19702505 -0.19702505 ... -0.21742004 -0.21742004
  -0.21742004]], shape=(65536, 40), dtype=float32)
------------------------------ 44 ------------------------------
[INFO]: embedding_vector:
 tf.Tensor(
[[-0.20941475 -0.20941475 -0.20941475 ... -0.22879735 -0.22879735
  -0.22879735]
 [-0.21896069 -0.21896069 -0.21896069 ... -0.23172814 -0.23172814
  -0.23172814]
 [-0.21667118 -0.21667118 -0.21667118 ... -0.21302357 -0.21302357
  -0.21302357]
 ...
 [-0.20077366 -0.20

In [13]:
# train SOK Demo Model, this command will print each iteration's embedding vector 
sok_results = test_sok_demo(args, init_tensors, *random_samples)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')
You are using the plugin with MirroredStrategy.
2021-12-07 04:48:48.852528: I sparse_operation_kit/kit_cc/kit_cc_infra/src/resources/manager.cc:99] Mapping from local_replica_id to device_id:
2021-12-07 04:48:48.852528: I sparse_operation_kit/kit_cc/kit_cc_infra/src/resources/manager.cc:101] 0 -> 0
2021-12-07 04:48:48.852528: I sparse_operation_kit/kit_cc/kit_cc_infra/src/resources/manager.cc:101] 1 -> 1
2021-12-07 04:48:48.852528: I sparse_operation_kit/kit_cc/kit_cc_infra/src/resources/manager.cc:101] 2 -> 2
2021-12-07 04:48:48.852528: I sparse_operation_

------------------------------ step  0 ------------------------------
INFO:tensorflow:batch_all_reduce: 2 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 2 all-reduces with algorithm = nccl, num_packs = 1
[INFO]: embedding_vector
 PerReplica:{
  0: tf.Tensor(
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]], shape=(8192, 40), dtype=float32),
  1: tf.Tensor(
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]], shape=(8192, 40), dtype=float32),
  2: tf.Tensor(
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]], shape=(8192, 40), dtype=float32),
  3: tf.Tensor(
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1.

------------------------------ step  3 ------------------------------
[INFO]: embedding_vector
 PerReplica:{
  0: tf.Tensor(
[[0.70422626 0.70422626 0.70422626 ... 0.7052028  0.7052028  0.7052028 ]
 [0.7036482  0.7036482  0.7036482  ... 0.7032463  0.7032463  0.7032463 ]
 [0.701295   0.701295   0.701295   ... 0.7022344  0.7022344  0.7022344 ]
 ...
 [0.7025738  0.7025738  0.7025738  ... 0.7037034  0.7037034  0.7037034 ]
 [0.7022596  0.7022596  0.7022596  ... 0.7078545  0.7078545  0.7078545 ]
 [0.70445216 0.70445216 0.70445216 ... 0.704602   0.704602   0.704602  ]], shape=(8192, 40), dtype=float32),
  1: tf.Tensor(
[[0.7023215  0.7023215  0.7023215  ... 0.70324075 0.70324075 0.70324075]
 [0.702116   0.702116   0.702116   ... 0.7016717  0.7016717  0.7016717 ]
 [0.7035055  0.7035055  0.7035055  ... 0.7026157  0.7026157  0.7026157 ]
 ...
 [0.7045913  0.7045913  0.7045913  ... 0.70173776 0.70173776 0.70173776]
 [0.702466   0.702466   0.702466   ... 0.7023295  0.7023295  0.7023295 ]
 [0.702456

------------------------------ step  6 ------------------------------
[INFO]: embedding_vector
 PerReplica:{
  0: tf.Tensor(
[[0.42153543 0.42153543 0.42153543 ... 0.42519388 0.42519388 0.42519388]
 [0.4170929  0.4170929  0.4170929  ... 0.41547838 0.41547838 0.41547838]
 [0.4118647  0.4118647  0.4118647  ... 0.4137937  0.4137937  0.4137937 ]
 ...
 [0.41602445 0.41602445 0.41602445 ... 0.4223327  0.4223327  0.4223327 ]
 [0.41451913 0.41451913 0.41451913 ... 0.4199844  0.4199844  0.4199844 ]
 [0.41644222 0.41644222 0.41644222 ... 0.41922688 0.41922688 0.41922688]], shape=(8192, 40), dtype=float32),
  1: tf.Tensor(
[[0.4137619  0.4137619  0.4137619  ... 0.41629443 0.41629443 0.41629443]
 [0.4158877  0.4158877  0.4158877  ... 0.42044166 0.42044166 0.42044166]
 [0.4148597  0.4148597  0.4148597  ... 0.41802114 0.41802114 0.41802114]
 ...
 [0.41918486 0.41918486 0.41918486 ... 0.41373587 0.41373587 0.41373587]
 [0.41620752 0.41620752 0.41620752 ... 0.420725   0.420725   0.420725  ]
 [0.422563

------------------------------ step  9 ------------------------------
[INFO]: embedding_vector
 PerReplica:{
  0: tf.Tensor(
[[0.16157222 0.16157222 0.16157222 ... 0.16512923 0.16512923 0.16512923]
 [0.17077649 0.17077649 0.17077649 ... 0.16518483 0.16518483 0.16518483]
 [0.16769904 0.16769904 0.16769904 ... 0.1654256  0.1654256  0.1654256 ]
 ...
 [0.16069056 0.16069056 0.16069056 ... 0.16277106 0.16277106 0.16277106]
 [0.16398567 0.16398567 0.16398567 ... 0.17398588 0.17398588 0.17398588]
 [0.15237144 0.15237144 0.15237144 ... 0.16489398 0.16489398 0.16489398]], shape=(8192, 40), dtype=float32),
  1: tf.Tensor(
[[0.18537274 0.18537274 0.18537274 ... 0.17068884 0.17068884 0.17068884]
 [0.16184442 0.16184442 0.16184442 ... 0.16080418 0.16080418 0.16080418]
 [0.16586392 0.16586392 0.16586392 ... 0.1633567  0.1633567  0.1633567 ]
 ...
 [0.14759749 0.14759749 0.14759749 ... 0.1542436  0.1542436  0.1542436 ]
 [0.16890895 0.16890895 0.16890895 ... 0.15605208 0.15605208 0.15605208]
 [0.155914

...

------------------------------ step  40 ------------------------------
[INFO]: embedding_vector
 PerReplica:{
  0: tf.Tensor(
[[-0.16127078 -0.16127078 -0.16127078 ... -0.18304515 -0.18304515
  -0.18304515]
 [-0.1922871  -0.1922871  -0.1922871  ... -0.14632958 -0.14632958
  -0.14632958]
 [-0.1974465  -0.1974465  -0.1974465  ... -0.15940674 -0.15940674
  -0.15940674]
 ...
 [-0.17839476 -0.17839476 -0.17839476 ... -0.23484176 -0.23484176
  -0.23484176]
 [-0.1537202  -0.1537202  -0.1537202  ... -0.17789915 -0.17789915
  -0.17789915]
 [-0.16126381 -0.16126381 -0.16126381 ... -0.19991472 -0.19991472
  -0.19991472]], shape=(8192, 40), dtype=float32),
  1: tf.Tensor(
[[-0.15530169 -0.15530169 -0.15530169 ... -0.15739058 -0.15739058
  -0.15739058]
 [-0.18126467 -0.18126467 -0.18126467 ... -0.1831001  -0.1831001
  -0.1831001 ]
 [-0.16429624 -0.16429624 -0.16429624 ... -0.1760344  -0.1760344
  -0.1760344 ]
 ...
 [-0.15476167 -0.15476167 -0.15476167 ... -0.17273518 -0.17273518
  -0.17273518]
 [-0

------------------------------ step  42 ------------------------------
[INFO]: embedding_vector
 PerReplica:{
  0: tf.Tensor(
[[-0.20266193 -0.20266193 -0.20266193 ... -0.17308879 -0.17308879
  -0.17308879]
 [-0.16007192 -0.16007192 -0.16007192 ... -0.20874861 -0.20874861
  -0.20874861]
 [-0.22338642 -0.22338642 -0.22338642 ... -0.19393001 -0.19393001
  -0.19393001]
 ...
 [-0.18364424 -0.18364424 -0.18364424 ... -0.21327041 -0.21327041
  -0.21327041]
 [-0.19914481 -0.19914481 -0.19914481 ... -0.19567479 -0.19567479
  -0.19567479]
 [-0.26409876 -0.26409876 -0.26409876 ... -0.20579208 -0.20579208
  -0.20579208]], shape=(8192, 40), dtype=float32),
  1: tf.Tensor(
[[-0.20218132 -0.20218132 -0.20218132 ... -0.21695873 -0.21695873
  -0.21695873]
 [-0.20778793 -0.20778793 -0.20778793 ... -0.20841631 -0.20841631
  -0.20841631]
 [-0.18760791 -0.18760791 -0.18760791 ... -0.2056439  -0.2056439
  -0.2056439 ]
 ...
 [-0.23459375 -0.23459375 -0.23459375 ... -0.19046655 -0.19046655
  -0.19046655]
 [-

------------------------------ step  44 ------------------------------
[INFO]: embedding_vector
 PerReplica:{
  0: tf.Tensor(
[[-0.20941171 -0.20941171 -0.20941171 ... -0.22879487 -0.22879487
  -0.22879487]
 [-0.21895784 -0.21895784 -0.21895784 ... -0.23172522 -0.23172522
  -0.23172522]
 [-0.21666765 -0.21666765 -0.21666765 ... -0.21301958 -0.21301958
  -0.21301958]
 ...
 [-0.21469995 -0.21469995 -0.21469995 ... -0.21231383 -0.21231383
  -0.21231383]
 [-0.21488464 -0.21488464 -0.21488464 ... -0.12033444 -0.12033444
  -0.12033444]
 [-0.20021415 -0.20021415 -0.20021415 ... -0.22670025 -0.22670025
  -0.22670025]], shape=(8192, 40), dtype=float32),
  1: tf.Tensor(
[[-0.23484477 -0.23484477 -0.23484477 ... -0.18834177 -0.18834177
  -0.18834177]
 [-0.21303031 -0.21303031 -0.21303031 ... -0.22244355 -0.22244355
  -0.22244355]
 [-0.2199068  -0.2199068  -0.2199068  ... -0.22984855 -0.22984855
  -0.22984855]
 ...
 [-0.20269692 -0.20269692 -0.20269692 ... -0.19387221 -0.19387221
  -0.19387221]
 [

------------------------------ step  46 ------------------------------
[INFO]: embedding_vector
 PerReplica:{
  0: tf.Tensor(
[[-0.23491383 -0.23491383 -0.23491383 ... -0.24627867 -0.24627867
  -0.24627867]
 [-0.23250745 -0.23250745 -0.23250745 ... -0.21174543 -0.21174543
  -0.21174543]
 [-0.20785967 -0.20785967 -0.20785967 ... -0.2619608  -0.2619608
  -0.2619608 ]
 ...
 [-0.24293315 -0.24293315 -0.24293315 ... -0.25826    -0.25826
  -0.25826   ]
 [-0.23931171 -0.23931171 -0.23931171 ... -0.20098595 -0.20098595
  -0.20098595]
 [-0.22243682 -0.22243682 -0.22243682 ... -0.24403623 -0.24403623
  -0.24403623]], shape=(8192, 40), dtype=float32),
  1: tf.Tensor(
[[-0.2253752  -0.2253752  -0.2253752  ... -0.23098066 -0.23098066
  -0.23098066]
 [-0.2145882  -0.2145882  -0.2145882  ... -0.22161697 -0.22161697
  -0.22161697]
 [-0.24860863 -0.24860863 -0.24860863 ... -0.23111928 -0.23111928
  -0.23111928]
 ...
 [-0.22765952 -0.22765952 -0.22765952 ... -0.24573767 -0.24573767
  -0.24573767]
 [-0.2

------------------------------ step  48 ------------------------------
[INFO]: embedding_vector
 PerReplica:{
  0: tf.Tensor(
[[-0.24711709 -0.24711709 -0.24711709 ... -0.28307924 -0.28307924
  -0.28307924]
 [-0.2479815  -0.2479815  -0.2479815  ... -0.2596137  -0.2596137
  -0.2596137 ]
 [-0.23282254 -0.23282254 -0.23282254 ... -0.24117479 -0.24117479
  -0.24117479]
 ...
 [-0.27805972 -0.27805972 -0.27805972 ... -0.24752554 -0.24752554
  -0.24752554]
 [-0.2415568  -0.2415568  -0.2415568  ... -0.2516385  -0.2516385
  -0.2516385 ]
 [-0.2403656  -0.2403656  -0.2403656  ... -0.24829827 -0.24829827
  -0.24829827]], shape=(8192, 40), dtype=float32),
  1: tf.Tensor(
[[-0.24093132 -0.24093132 -0.24093132 ... -0.2616806  -0.2616806
  -0.2616806 ]
 [-0.2341744  -0.2341744  -0.2341744  ... -0.23874548 -0.23874548
  -0.23874548]
 [-0.23052762 -0.23052762 -0.23052762 ... -0.23824754 -0.23824754
  -0.23824754]
 ...
 [-0.2503054  -0.2503054  -0.2503054  ... -0.2665841  -0.2665841
  -0.2665841 ]
 [-0.2

Finally, check the consistency of the embedding vectors obtained from TensorFlow ans SOK.

In [14]:
if (len(sok_results) != len(tf_results)):
    raise ValueError("The length of sok results is not equal to that of TensorFlow.")
if (len(tf_results) != args["iter_num"]):
    raise ValueError("The length of embedding vectors: %d is not equal to iteration number: %d."
                    %(len(tf_results), args["iter_num"]))
    
for i, sok_vector in enumerate(sok_results):
    if args["gpu_num"] != 1:
        sok_vector = tf.stack(sok_vector.values, axis=0)
    tf.debugging.assert_near(tf.reshape(sok_vector,
                                        shape=[-1, tf.shape(sok_vector)[-1]]),
                             tf_results[i],
                             atol=1e-4,
                             rtol=1e-4,
                             message="The embedding vectors obtained from TF and SOK vary in iteration: %d" %i)
    
# if no exception, then the embedding vectors for all iterations are consistent.
print(("\n[INFO]: With MirroredStrategy, when %d GPUs are used, the embedding vectors obtained from TensorFlow" 
       "and SOK are consistent for %d iterations") 
      %(args["gpu_num"], args["iter_num"]))


[INFO]: With MirroredStrategy, when 8 GPUs are used, the embedding vectors obtained from TensorFlowand SOK are consistent for 50 iterations


### Multi-node, Multi-GPUs synchronized training ###

**The jupyter notebook kernel must be restarted!!**

Firstly, specify hyper parameters

In [1]:
%reset -f

args = dict()

args["iter_num"] = 50                             # the number of training iteration
args["max_vocabulary_size_per_gpu"] = 1024
args["slot_num"] = 10                             # the number of feature fields in this embedding layer
args["max_nnz"] = 4                               # the maximum number of valid features in each slot
args["embedding_vec_size"] = 4                    # the dimension of embedding vectors
args["combiner"] = "mean"                         # the reduction combiner used intra slots, it can be [mean, sum]
args["global_batch_size"] = 65536                 # the globally batchsize for all GPUs
args["optimizer"] = "plugin_adam"                 # the optimizer used for training, it can be [plugin_adam, adam, sgd]
args["ips"] = ["localhost", "localhost"]          # specify the ip addr of each node. Here we use different GPUs to 
                                                  # simulate multi-node with single-node    
args["worker_num"] = len(args["ips"])             # the number of workers in synchronized training

In [2]:
import sys, os, json
import sparse_operation_kit as sok
import tensorflow as tf
import numpy as np
# import utility python script
sys.path.append("../unit_test/test_scripts/tf2/")
import utils

[INFO]: sparse_operation_kit is imported


In [3]:
total_gpu_num = utils.get_local_gpu_count()
print("[INFO]: There are %d GPUs in total" %total_gpu_num)
if (total_gpu_num % args["worker_num"] != 0):
    raise RuntimeError("total_gpu_num:%d is not divisible by workers_num: %d" %(total_gpu_num, args["worker_num"]))
    
per_worker_gpu_num = total_gpu_num // args["worker_num"]
args["local_gpu_num"] = per_worker_gpu_num # the number of avaiable GPUs in each process

[INFO]: There are 8 GPUs in total


Secondly, define DNN model using Tensorflow and SOK.

In [4]:
class TfDemo(tf.keras.models.Model):
    def __init__(self, 
                 init_tensors, 
                 combiner, 
                 global_batch_size,
                 slot_num, 
                 embedding_vec_size,
                 **kwargs):
        super(TfDemo, self).__init__(**kwargs)
        self.combiner = combiner
        self.global_batch_size = global_batch_size
        self.slot_num = slot_num
        self.embedding_vec_size = embedding_vec_size

        self.init_tensors = init_tensors
        self.params = tf.Variable(initial_value=tf.concat(self.init_tensors, axis=0))

        self.dense_layer = tf.keras.layers.Dense(units=1, activation=None,
                                                 kernel_initializer="ones",
                                                 bias_initializer="zeros")

    def call(self, inputs, training=True):
        # [batchsize * slot_num, embedding_vec_size]
        embedding_vector = tf.nn.embedding_lookup_sparse(params=self.params, sp_ids=inputs,
                                                        sp_weights=None, combiner=self.combiner)

        # [batchsize, slot_num * embedding_vec_size]
        embedding_vector = tf.reshape(embedding_vector, shape=[self.global_batch_size, self.slot_num * self.embedding_vec_size])
        logit = self.dense_layer(embedding_vector)
        return logit, embedding_vector

In [5]:
class SOKDemo(tf.keras.models.Model):
    def __init__(self,
                 combiner,
                 max_vocabulary_size_per_gpu,
                 slot_num,
                 max_nnz,
                 embedding_vec_size, 
                 **kwargs):
        super(SOKDemo, self).__init__(**kwargs)

        self.combiner = combiner
        self.max_vocabulary_size_per_gpu = max_vocabulary_size_per_gpu
        self.slot_num = slot_num
        self.max_nnz = max_nnz
        self.embedding_vec_size = embedding_vec_size

        self.embedding_layer = sok.DistributedEmbedding(combiner=self.combiner,
                                                           max_vocabulary_size_per_gpu=self.max_vocabulary_size_per_gpu,
                                                           embedding_vec_size=self.embedding_vec_size,
                                                           slot_num=self.slot_num,
                                                           max_nnz=self.max_nnz)

        self.dense_layer = tf.keras.layers.Dense(units=1, activation=None,
                                                 kernel_initializer="ones",
                                                 bias_initializer="zeros")

    def call(self, inputs, training=True):
        # [batchsize, slot_num, embedding_vec_size]
        embedding_vector = self.embedding_layer(inputs, training=training)
        # [batchsize, slot_num * embedding_vec_size]
        embedding_vector = tf.reshape(embedding_vector, shape=[-1, self.slot_num * self.embedding_vec_size])
        # [batchsize, 1]
        logit = self.dense_layer(embedding_vector)
        return logit, embedding_vector

In [6]:
def test_tf_demo(args, init_tensors, *random_samples):
    dataset = utils.tf_dataset(*random_samples, batchsize=args["global_batch_size"], to_sparse_tensor=True, repeat=1)

    loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)

    tf_demo = TfDemo(init_tensors, args["combiner"], args["global_batch_size"], 
                     args["slot_num"], args["embedding_vec_size"])

    optimizer = utils.get_dense_optimizer(args["optimizer"])(learning_rate=0.1)

    @tf.function
    def _train_step(inputs, labels):
        with tf.GradientTape() as tape:
            logit, embedding_vector = tf_demo(inputs, training=True)
            loss = loss_fn(labels, logit)
        grads = tape.gradient(loss, tf_demo.trainable_variables)
        optimizer.apply_gradients(zip(grads, tf_demo.trainable_variables))
        return logit, embedding_vector

    tf_results = list()

    for i, (sparse_tensors, labels) in enumerate(dataset):
        print("-"*30, str(i), "-"*30)
        logit, embedding_vector = _train_step(sparse_tensors, labels)
        print("[INFO]: embedding_vector:\n", embedding_vector)
        tf_results.append(embedding_vector)

    return tf_results

Thirdly, define multi-node training loop for SOK.

In [7]:
def test_sok_demo(args, task_id, init_tensors, *random_samples):
    physical_devices = tf.config.list_physical_devices('GPU')
    print("[INFO]: physical_devices on task %d:" %task_id, physical_devices)
    
    port = 12345
    os.environ["TF_CONFIG"] = json.dumps({
        'cluster': {"worker": [args["ips"][i] + ":" + str(port + i) for i in range(args["worker_num"])] },
        'task': {"type": 'worker', "index": task_id}
    })
    strategy = tf.distribute.MultiWorkerMirroredStrategy()
    with strategy.scope():
        sok.Init(global_batch_size=args["global_batch_size"])

        sok_demo = SOKDemo(combiner=args["combiner"], 
                            max_vocabulary_size_per_gpu=args["max_vocabulary_size_per_gpu"],
                            slot_num=args["slot_num"], max_nnz=args["max_nnz"],
                            embedding_vec_size=args["embedding_vec_size"])

        emb_opt = utils.get_embedding_optimizer(args["optimizer"])(learning_rate=0.1)
        dense_opt = utils.get_dense_optimizer(args["optimizer"])(learning_rate=0.1)
    
    sok_saver = sok.Saver()
    sok_saver.load_embedding_values(sok_demo.embedding_layer.embedding_variable, init_tensors)

    loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
    def _replica_loss(labels, logits):
        loss = loss_fn(labels, logits)
        return tf.nn.compute_average_loss(loss, global_batch_size=args["global_batch_size"])

    @tf.function
    def _train_step(inputs, labels):
        with tf.GradientTape() as tape:
            logit, embedding_vector = sok_demo(inputs, training=True)
            loss = _replica_loss(labels, logit)
        embedding_variables, other_variable = sok.split_embedding_variable_from_others(sok_demo.trainable_variables)
        grads, emb_grads = tape.gradient(loss, [other_variable, embedding_variables])
        if "plugin" not in args["optimizer"]:
            with sok.OptimizerScope(embedding_variables):
                emb_opt.apply_gradients(zip(emb_grads, embedding_variables),
                                        experimental_aggregate_gradients=False)
        else:
            emb_opt.apply_gradients(zip(emb_grads, embedding_variables),
                                    experimental_aggregate_gradients=False)
        dense_opt.apply_gradients(zip(grads, other_variable))
        return logit, embedding_vector

    sok_results = list()

    def _dataset_fn(input_context):
        replica_batch_size = input_context.get_per_replica_batch_size(args["global_batch_size"])
        dataset = utils.tf_dataset(*random_samples, batchsize=replica_batch_size, to_sparse_tensor=True, repeat=1)
        # because each worker has its own data source, so that no need to shard the dataset.
        return dataset

    dataset = strategy.distribute_datasets_from_function(_dataset_fn)

    for i, (sparse_tensors, replica_labels) in enumerate(dataset):
        print("-" * 30, "step ", str(i), "-" * 30)
        logit, embedding_vector = strategy.run(_train_step, args=(sparse_tensors, replica_labels))
        print("[INFO]: embedding_vector\n", embedding_vector)
        sok_results.append(embedding_vector)

    return sok_results

Fourthly, define subprocess work function to simulate multi-node synchronized training

In [8]:
def compare_sok_with_tf(args, task_id):
    if (args["global_batch_size"] % args["local_gpu_num"] != 0):
        raise ValueError("global_batch_size: %d is not divisible by local_gpu_num: %d"
                            %(args["global_batch_size"], args["local_gpu_num"]))
    if (args["global_batch_size"] % args["worker_num"] != 0):
        raise ValueError("global_batch_size: %d is not divisible by worker_num: %d"
                            %(args["global_batch_size"], args["worker_num"]))

    # each worker generate different dataset
    worker_batch_size = args["global_batch_size"] // args["worker_num"]
    random_samples_local = utils.generate_random_samples(num_of_samples=worker_batch_size * args["iter_num"],
                                                         vocabulary_size=args["local_gpu_num"] * args["max_vocabulary_size_per_gpu"] * args["worker_num"],
                                                         slot_num=args["slot_num"],
                                                         max_nnz=args["max_nnz"])
    utils.save_to_file(r"./random_samples_" + str(task_id) + r".file", *random_samples_local)

    # each worker generate same init tensors, because each worker will do the filtering by itself.
    init_tensors = utils.get_ones_tensor(max_vocab_size_per_gpu=args["max_vocabulary_size_per_gpu"],
                                            embedding_vec_size=args["embedding_vec_size"],
                                            num=args["local_gpu_num"] * args["worker_num"])

    sok_results_local = test_sok_demo(args, task_id, init_tensors, *random_samples_local)
    # save the forward embedding vector from different worker to file
    utils.save_to_file(r"./sok_embedding_vectors_" + str(task_id) + r".file", *sok_results_local)

    # aggregate dataset from different worker
    dataset_filenames = [r"./random_samples_" + str(task_id) + r".file"
                         for task_id in range(args["worker_num"])]
    random_samples_total = [list() for _ in range(args["iter_num"])]
    random_labels_total = [list() for _ in range(args["iter_num"])]
    local_batch_size = args["global_batch_size"] // args["worker_num"]
    for work_id in range(args["worker_num"]):
        samples, labels = utils.restore_from_file(dataset_filenames[work_id])
        for i in range(args["iter_num"]):
            random_samples_total[i].extend(samples[i * local_batch_size : (i + 1) * local_batch_size])
            random_labels_total[i].extend(labels[i * local_batch_size : (i + 1) * local_batch_size])
    random_samples_total = np.concatenate(random_samples_total, axis=0)
    random_labels_total = np.concatenate(random_labels_total, axis=0)

    tf_results = test_tf_demo(args, init_tensors, random_samples_total, random_labels_total)

    # aggregate forward embedding vector from different worker
    sok_results_filenames = [r"./sok_embedding_vectors_" + str(task_id) + r".file"
                             for task_id in range(args["worker_num"])]
    sok_results_total = list()
    for file_name in sok_results_filenames:
        sok_results_local = utils.restore_from_file(file_name)
        sok_results_total.append(sok_results_local)

    if (len(sok_results_total[0]) != len(tf_results)):
        raise ValueError("The length of results obtained from sok: %d is not equal to that of tensorflow: %d."
                        %(len(sok_results_total[0]), len(tf_results)))
    if (len(tf_results) != args["iter_num"]):
        raise ValueError("The length of embedding vectors: %d is not equal to iteration number: %d."
                         %(len(tf_results), args["iter_num"]))

    # for i, sok_vector in enumerate(sok_results_total):
    for i in range(args["iter_num"]):
        if args["local_gpu_num"] != 1:
            sok_vector = tf.concat([tf.concat(sok_results_total[task_id][i].values, axis=0)
                                    for task_id in range(args["worker_num"])], axis=0)
        else:
            sok_vector = tf.concat([sok_results_total[task_id][i]
                                    for task_id in range(args["worker_num"])], axis=0)
        tf.debugging.assert_near(tf.reshape(sok_vector, 
                                            shape=[-1, tf.shape(sok_vector)[-1]]),
                                 tf_results[i],
                                 atol=1e-4,
                                 rtol=1e-4)

    print(("\n[INFO]: With MultiWorkerMirroredStrategy, when %d GPUs are used for each node and %d GPUs in total, "
           "the embedding vectors obtained from TensorFlow and SOK are consistent for %d iterations")
          %(args["local_gpu_num"], args["local_gpu_num"] * args["worker_num"], args["iter_num"]))

Fifthly, create sub CPU processes to simulate multi-node synchronized training 

In [9]:
from multiprocessing import Process

processes = list()
for task_id in range(args["worker_num"]):
    available_gpus = ",".join([str(per_worker_gpu_num * task_id + i)
                              for i in range(per_worker_gpu_num)])
    print("[INFO]: on task: %d, its avaiable GPUs are: %s" %(task_id, available_gpus))
    
    os.environ["CUDA_VISIBLE_DEVICES"] = available_gpus
    process = Process(target=compare_sok_with_tf, args=(args, task_id))
    process.start()
    processes.append(process)
    
    
for process in processes:
    if process.is_alive():
        process.join()

[INFO]: on task: 0, its avaiable GPUs are: 0,1,2,3
[INFO]: on task: 1, its avaiable GPUs are: 4,5,6,7
[INFO]: begin to generate random samples
[INFO]: begin to generate random samples
[INFO]: generated random samples
[INFO]: generated random samples
[INFO]: dumpped items to file ./random_samples_1.file
[INFO]: physical_devices on task 1: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]
[INFO]: dumpped items to file ./random_samples_0.file
[INFO]: physical_devices on task 0: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]
INFO:tensorflow:Enabled multi-worker col

2021-12-07 04:50:11.099010: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2021-12-07 04:50:11.099107: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14131 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:85:00.0, compute capability: 7.0
2021-12-07 04:50:11.101215: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2021-12-07 04:50:11.101268: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14631 MB memory:  -> device: 1, name: Tesla V100-SXM2-16GB, pci bus id: 0000:86:00.0, compute capability: 7.0
2021-12-07 04:50:11.103196: W tensorflow/cor

INFO:tensorflow:Cluster is ready.
INFO:tensorflow:Cluster is ready.
INFO:tensorflow:MultiWorkerMirroredStrategy with cluster_spec = {'worker': ['localhost:12345', 'localhost:12346']}, task_type = 'worker', task_id = 1, num_workers = 2, local_devices = ('/job:worker/task:1/device:GPU:0', '/job:worker/task:1/device:GPU:1', '/job:worker/task:1/device:GPU:2', '/job:worker/task:1/device:GPU:3'), communication = CommunicationImplementation.AUTO
INFO:tensorflow:MultiWorkerMirroredStrategy with cluster_spec = {'worker': ['localhost:12345', 'localhost:12346']}, task_type = 'worker', task_id = 0, num_workers = 2, local_devices = ('/job:worker/task:0/device:GPU:0', '/job:worker/task:0/device:GPU:1', '/job:worker/task:0/device:GPU:2', '/job:worker/task:0/device:GPU:3'), communication = CommunicationImplementation.AUTO


2021-12-07 04:50:12.670152: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-07 04:50:12.713944: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: cond/branch_executed/_6
2021-12-07 04:50:13.649949: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-07 04:50:13.850567: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: replica_3/cond_1/branch_executed/_55


2021-12-07 04:50:13.852613: I sparse_operation_kit/kit_cc/kit_cc_infra/src/resources/manager.cc:99] Mapping from local_replica_id to device_id:
2021-12-07 04:50:13.852613: I sparse_operation_kit/kit_cc/kit_cc_infra/src/resources/manager.cc:101] 0 -> 0
2021-12-07 04:50:13.852613: I sparse_operation_kit/kit_cc/kit_cc_infra/src/resources/manager.cc:101] 1 -> 1
2021-12-07 04:50:13.852613: I sparse_operation_kit/kit_cc/kit_cc_infra/src/resources/manager.cc:101] 2 -> 2
2021-12-07 04:50:13.852613: I sparse_operation_kit/kit_cc/kit_cc_infra/src/resources/manager.cc:101] 3 -> 3
2021-12-07 04:50:13.852613: I sparse_operation_kit/kit_cc/kit_cc_infra/src/resources/manager.cc:77] Global seed is 783670919
2021-12-07 04:50:13.852613: I sparse_operation_kit/kit_cc/kit_cc_infra/src/resources/manager.cc:78] Local GPU Count: 4
2021-12-07 04:50:13.852613: I sparse_operation_kit/kit_cc/kit_cc_infra/src/resources/manager.cc:79] Global GPU Count: 8
2021-12-07 04:50:13.852613: I sparse_operation_kit/kit_cc/ki

...

------------------------------ 21 ------------------------------
------------------------------ 23 ------------------------------
[INFO]: embedding_vector:
 tf.Tensor(
[[-0.319013   -0.319013   -0.319013   ... -0.31606743 -0.31606743
  -0.31606743]
 [-0.3031985  -0.3031985  -0.3031985  ... -0.29684377 -0.29684377
  -0.29684377]
 [-0.3093907  -0.3093907  -0.3093907  ... -0.3269143  -0.3269143
  -0.3269143 ]
 ...
 [-0.31462306 -0.31462306 -0.31462306 ... -0.3337177  -0.3337177
  -0.3337177 ]
 [-0.30985603 -0.30985603 -0.30985603 ... -0.3014966  -0.3014966
  -0.3014966 ]
 [-0.30994752 -0.30994752 -0.30994752 ... -0.33786583 -0.33786583
  -0.33786583]], shape=(65536, 40), dtype=float32)
[INFO]: embedding_vector:
 tf.Tensor(
[[-0.3010753  -0.3010753  -0.3010753  ... -0.26137453 -0.26137453
  -0.26137453]
 [-0.2920072  -0.2920072  -0.2920072  ... -0.26624942 -0.26624942
  -0.26624942]
 [-0.29443598 -0.29443598 -0.29443598 ... -0.31122905 -0.31122905
  -0.31122905]
 ...
 [-0.2656673  -0.26566

  -0.18044093]], shape=(65536, 40), dtype=float32)
[INFO]: embedding_vector:
------------------------------  30 tf.Tensor(
[[-0.20522626 -0.20522626 -0.20522626 ... -0.17386943 -0.17386943
  -0.17386943]
 [-0.2058292  -0.2058292  -0.2058292  ... -0.17246705 -0.17246705
  -0.17246705]
 [-0.21884426 -0.21884426 -0.21884426 ... -0.21649659 -0.21649659
  -0.21649659]
 ...
 [-0.21075189 -0.21075189 -0.21075189 ... -0.21303983 -0.21303983
  -0.21303983]
 [-0.23248424 -0.23248424 -0.23248424 ... -0.22775568 -0.22775568
  -0.22775568]
 [-0.22323845 -0.22323845 -0.22323845 ... -0.22112778 -0.22112778
  -0.22112778]], shape=(65536, 40), dtype=float32)------------------------------

------------------------------ 28 ------------------------------
[INFO]: embedding_vector:
 tf.Tensor(
[[-0.19175145 -0.19175145 -0.19175145 ... -0.16623998 -0.16623998
  -0.16623998]
 [-0.16289337 -0.16289337 -0.16289337 ... -0.1725828  -0.1725828
  -0.1725828 ]
 [-0.16131778 -0.16131778 -0.16131778 ... -0.16490965 -

  -0.11739247]], shape=(65536, 40), dtype=float32)
[INFO]: embedding_vector:
 tf.Tensor(
[[-0.15180266 -0.15180266 -0.15180266 ... -0.15956396 -0.15956396
  -0.15956396]
 [-0.18056567 -0.18056567 -0.18056567 ... -0.11601666 -0.11601666
  -0.11601666]
 [-0.10885726 -0.10885726 -0.10885726 ... -0.17777488 -0.17777488
  -0.17777488]
 ...
 [-0.12706445 -0.12706445 -0.12706445 ... -0.12976485 -0.12976485
  -0.12976485]
 [-0.12794422 -0.12794422 -0.12794422 ... -0.13762495 -0.13762495
  -0.13762495]
 [-0.11606186 -0.11606186 -0.11606186 ... -0.1576132  -0.1576132
  -0.1576132 ]], shape=(65536, 40), dtype=float32)
------------------------------ 37 ------------------------------
------------------------------ 34 ------------------------------
[INFO]: embedding_vector:
 tf.Tensor(
[[-0.11994702 -0.11994702 -0.11994702 ... -0.16632766 -0.16632766
  -0.16632766]
 [-0.12410761 -0.12410761 -0.12410761 ... -0.17561823 -0.17561823
  -0.17561823]
 [-0.13765712 -0.13765712 -0.13765712 ... -0.15519011 -

  -0.15296361]], shape=(65536, 40), dtype=float32)
[INFO]: embedding_vector:
 tf.Tensor(
[[-0.21763018 -0.21763018 -0.21763018 ... -0.21754324 -0.21754324
  -0.21754324]
 [-0.19721556 -0.19721556 -0.19721556 ... -0.20997626 -0.20997626
  -0.20997626]
 [-0.18108685 -0.18108685 -0.18108685 ... -0.17802556 -0.17802556
  -0.17802556]
 ...
 [-0.17719774 -0.17719774 -0.17719774 ... -0.1929565  -0.1929565
  -0.1929565 ]
 [-0.20807157 -0.20807157 -0.20807157 ... -0.18814465 -0.18814465
  -0.18814465]
 [-0.21611315 -0.21611315 -0.21611315 ... -0.1911238  -0.1911238
  -0.1911238 ]], shape=(65536, 40), dtype=float32)
------------------------------ 40 ------------------------------
------------------------------ 44 ------------------------------
[INFO]: embedding_vector:
 tf.Tensor(
[[-0.1847917  -0.1847917  -0.1847917  ... -0.18993592 -0.18993592
  -0.18993592]
 [-0.17649817 -0.17649817 -0.17649817 ... -0.17080042 -0.17080042
  -0.17080042]
 [-0.18804726 -0.18804726 -0.18804726 ... -0.15337943 -0

------------------------------ 46 ------------------------------
[INFO]: embedding_vector:
 tf.Tensor(
[[-0.22544068 -0.22544068 -0.22544068 ... -0.23053782 -0.23053782
  -0.23053782]
 [-0.26636156 -0.26636156 -0.26636156 ... -0.24495119 -0.24495119
  -0.24495119]
 [-0.20872188 -0.20872188 -0.20872188 ... -0.25625733 -0.25625733
  -0.25625733]
 ...
 [-0.2380555  -0.2380555  -0.2380555  ... -0.26428485 -0.26428485
  -0.26428485]
 [-0.2280464  -0.2280464  -0.2280464  ... -0.22280021 -0.22280021
  -0.22280021]
 [-0.25092384 -0.25092384 -0.25092384 ... -0.2429116  -0.2429116
  -0.2429116 ]], shape=(65536, 40), dtype=float32)
------------------------------ 47 ------------------------------
[INFO] loadded from file ./sok_embedding_vectors_0.file
[INFO]: embedding_vector:
 tf.Tensor(
[[-0.22355214 -0.22355214 -0.22355214 ... -0.23591082 -0.23591082
  -0.23591082]
 [-0.27213818 -0.27213818 -0.27213818 ... -0.25591666 -0.25591666
  -0.25591666]
 [-0.27956653 -0.27956653 -0.27956653 ... -0.24207

If no exceptions and their embedding vectors are totally consistent, then a sentence similar to the following one will be printed.
```shell
"[INFO]: With MultiWorkerMirroredStrategy, when args["local_gpu_num"] GPUs are used for each node and total_gpu_num GPUs in total, the embedding vectors obtained from TensorFlow and SOK are consistent for 50 iterations"
```